# Модуль 18. Практическая работа

## Цели работы

- Попрактиковаться в предобработке текстовых данных (токенизация, лемматизация, векторизация).

- Построить две модели классификации с разными способами векторизации текстов (`CountVectorizer` и `TfIdfVectorizer`).
- Сравнить качество моделей.

## Что нужно сделать

1. Загрузите датасет из файла, приложенного к заданию (`m18_jokes_dataset.csv`).

1. Закодируйте целевую переменную с помощью `LabelEncoder`.
1. Разделите выборку на тренировочную и тестовую.
1. Используя знания из модуля, подготовьте функцию `preprocess`, которая:
- удаляет стоп-слова в соответствии со списком `nltk.corpus.stopwords`;
- приводит слова к нормальной форме с помощью `pymorphy2`.
5. Создайте экзепляры векторизаторов `CountVectorizer` и `TfIdfVectorizer`. В качестве функции предобработки данных оба должны использовать функцию `preprocess`.

1. Векторизуйте признаки тренировочной и тестовой выборок. Сохраните результаты `CountVectorizer` в `X_train_count` и `X_test_count`, а результаты `TfIdfVectorizer` – в `X_train_tfidf` и `X_test_tfidf`.
1. Обучите две модели классификации, например `LinearSVC`.
1. Оцените работу обеих моделей на тестовой выборке, используя `f1_score` с методом усреднения `macro` (параметр `average='macro'`) – это арифметическое среднее f1-метрик, рассчитанных для каждого класса. 

## Что оценивается

- Все шаги выполнены правильно.

- Код решения соответствует референсному коду, который есть у куратора. 
- Обучены две модели, которые используют текстовые данные, обработанные разными векторизаторами (`CountVectorizer` и `TfIdfVectorizer`). Для обеих моделей вычислен усреднённый `f1_score` (метод усреднения — `macro`). 

## Информация о задаче

### Описание датасета

`m18_jokes_dataset.csv` – датасет, подготовленный на основе датасета, размещённого [на kaggle](https://www.kaggle.com/datasets/konstantinalbul/russian-jokes). Наш датасет содержит две колонки:

- **text** — текст анекдота (скорее всего, несмешного, но ваша задача не смеяться, а решить задачу классификации);

- **theme** — категория, к которой относится шутка. 

Датасет содержит данные для задачи многоклассовой классификации. В отличие от оригинального датасета, в нашем классов меньше, зато они идеально сбалансированы: в каждом ровно тысяча примеров. Так вам будет проще сконцентрироваться на текстовых данных.

### Порядок выполнения 

Напишите весь необходимый код в ячейках ниже.

### Рекомендации

- Обратите внимание, что модели sklearn (например, `LinearSVC`) умеют работать с разреженными матрицами (`scipy.sparse._csr.csr_matrix`), которые возвращают методы и `fit_transform` и `transform` векторизаторов `CountVectorizer` и `TfIdfVectorizer`.

- Когда подготавливаете векторизатор, вызывайте `fit_transform` на тренировочной выборке, а потом `fit` на тестовой, примерно так:
    > X_train_count = count_vectorizer.fit_transform(X_train)
    >
    > X_test_count = count_vectorizer.transform(X_test)

Затем используйте полученные результаты для обучения и контроля моделей `sklearn`.

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pymorphy2

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv("m18_jokes_dataset.csv")

df.head()

,theme,text
0,aforizmi,"Почему удар в спину наносят те, кого,как прави..."
1,aforizmi,Музы - пожалуй самые мудрые представительницы...
2,aforizmi,"Тяжело бухать всю ночь, особенно если ночь пол..."
3,aforizmi,ПОКУШЕНИЕ. НАЛИЧНОСТЬ.\r\n\r\n\r\n
4,aforizmi,"Когда медленно танцуешь, ничего не мешает...\r..."


In [3]:
# Датасет идеально сбалансирован
df.groupby("theme").size()

theme
aforizmi                 1000
meditsinskie             1000
narodnie                 1000
poshlie-i-intimnie       1000
pro-alkogolikov          1000
pro-armiu                1000
pro-detey                1000
pro-evreev               1000
pro-militsiyu            1000
pro-mugchin              1000
pro-novih-russkih        1000
pro-semyu                1000
pro-studentov            1000
pro-vovochku             1000
raznie                   1000
shkolnie-i-pro-shkolu    1000
tsitati                  1000
dtype: int64

In [4]:
label_encoder = LabelEncoder()
df["theme"] = label_encoder.fit_transform(df["theme"])
df.sample(5)

,theme,text
5374,5,"Ежемесячный журнал ""СОЛДАТ""\r\nСтатья по обуч..."
8695,8,181.медкомиссия обследует идиота:\r\n- ваше им...
12465,12,Два препода в столовке. Один что-то режет ножо...
6493,6,Воспитательница повела детей на экскуpсию на ...
9403,9,- Инспектор! Не мог я превысить скорость!- Как...


In [5]:
X, y = df["text"], df["theme"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
tokenizer = RegexpTokenizer("\w+")
russian_stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()


def preprocess(text):
    stemmed_words = []
    for word in tokenizer.tokenize(text):
        word = word.lower()
        if word not in russian_stopwords:
            stemmed_words.append(morph.parse(word)[0].normal_form)
    return " ".join(stemmed_words)

In [ ]:
count_vectorizer = CountVectorizer(preprocessor=preprocess)

tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocess)

In [8]:
%%time

X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

CPU times: total: 52.9 s
Wall time: 50.5 s


In [9]:
%%time

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

CPU times: total: 54 s
Wall time: 51.1 s


In [10]:
svc_count = LinearSVC(max_iter=10000)

svc_count.fit(X_train_count, y_train)

y_pred_count = svc_count.predict(X_test_count)

print(classification_report(y_test, y_pred_count))

              precision    recall  f1-score   support

           0       0.38      0.71      0.50       198
           1       0.55      0.58      0.57       182
           2       0.45      0.32      0.37       204
           3       0.38      0.36      0.37       195
           4       0.90      0.88      0.89       184
           5       0.64      0.66      0.65       209
           6       0.93      0.94      0.94       202
           7       0.77      0.81      0.79       208
           8       0.92      0.67      0.78       193
           9       0.29      0.34      0.32       209
          10       0.82      0.83      0.82       203
          11       0.42      0.49      0.45       183
          12       0.70      0.66      0.68       211
          13       0.87      0.88      0.87       200
          14       0.15      0.15      0.15       213
          15       0.97      0.83      0.90       207
          16       0.63      0.33      0.43       199

    accuracy              

In [11]:
f1_score(y_test, y_pred_count, average='macro')

0.6163008338524399

In [12]:
%%time

svc_tfidf = LinearSVC(max_iter=10000)

svc_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = svc_tfidf.predict(X_test_tfidf)

classification_report(y_test, y_pred_tfidf)

CPU times: total: 422 ms
Wall time: 382 ms


'              precision    recall  f1-score   support\n\n           0       0.44      0.49      0.47       198\n           1       0.59      0.72      0.65       182\n           2       0.51      0.46      0.49       204\n           3       0.42      0.48      0.45       195\n           4       0.86      0.90      0.88       184\n           5       0.72      0.74      0.73       209\n           6       0.92      0.93      0.92       202\n           7       0.77      0.86      0.81       208\n           8       0.86      0.69      0.76       193\n           9       0.32      0.31      0.32       209\n          10       0.80      0.88      0.84       203\n          11       0.43      0.49      0.46       183\n          12       0.72      0.72      0.72       211\n          13       0.83      0.88      0.86       200\n          14       0.15      0.11      0.13       213\n          15       0.98      0.83      0.90       207\n          16       0.56      0.46      0.51       199\n\n    a

In [13]:
f1_score(y_test, y_pred_tfidf, average='macro')

0.6397111346120032